In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_json("final_temp.json")
df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street
100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",low,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,[https://photos.renthop.com/2/6930771_7e3622b6...,3600,210 West 107th Street
100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",low,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,[https://photos.renthop.com/2/6867392_b18283f6...,5645,155 West 21st Street
100099,2.0,2,39661b8ce46a8d71461497819f67c279,2016-04-06 06:27:51,Stunning two bedroom / two bathroom corner apa...,West 31st Street,"[Roof Deck, Dining Room, Elevator, Laundry in ...",low,40.7471,6832226,-73.9867,ad3d8ddc52c7e0859b5c6c7f7949c3bd,[https://photos.renthop.com/2/6832226_d974f188...,6500,9 West 31st Street


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9667 entries, 10000 to 99980
Data columns (total 15 columns):
bathrooms          9667 non-null float64
bedrooms           9667 non-null int64
building_id        9667 non-null object
created            9667 non-null object
description        9667 non-null object
display_address    9667 non-null object
features           9667 non-null object
interest_level     9667 non-null object
latitude           9667 non-null float64
listing_id         9667 non-null int64
longitude          9667 non-null float64
manager_id         9667 non-null object
photos             9667 non-null object
price              9667 non-null int64
street_address     9667 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 1.2+ MB


In [3]:
train_df = pd.read_json("data/train.json")
test_df = pd.read_json("data/test.json")

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9871 entries, 10000 to 99980
Data columns (total 14 columns):
bathrooms          9871 non-null float64
bedrooms           9871 non-null int64
building_id        9871 non-null object
created            9871 non-null object
description        9871 non-null object
display_address    9871 non-null object
features           9871 non-null object
latitude           9871 non-null float64
listing_id         9871 non-null int64
longitude          9871 non-null float64
manager_id         9871 non-null object
photos             9871 non-null object
price              9871 non-null int64
street_address     9871 non-null object
dtypes: float64(3), int64(3), object(8)
memory usage: 1.1+ MB


In [8]:
test_df = df
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9667 entries, 10000 to 99980
Data columns (total 15 columns):
bathrooms          9667 non-null float64
bedrooms           9667 non-null int64
building_id        9667 non-null object
created            9667 non-null object
description        9667 non-null object
display_address    9667 non-null object
features           9667 non-null object
interest_level     9667 non-null object
latitude           9667 non-null float64
listing_id         9667 non-null int64
longitude          9667 non-null float64
manager_id         9667 non-null object
photos             9667 non-null object
price              9667 non-null int64
street_address     9667 non-null object
dtypes: float64(3), int64(3), object(9)
memory usage: 1.2+ MB


In [10]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
def feature_engineering(df, flag, b_dict={}, m_dict={}):
    
    df["created"] = pd.to_datetime(df["created"])
    df["created_year"] = df["created"].dt.year
    df["created_month"] = df["created"].dt.month
    df["created_day"] = df["created"].dt.day
    df["created_hour"] = df["created"].dt.hour
    df["is_night"] = 1*(df["created_hour"] <= 7)
    df["created_weekday"] = df["created"].dt.weekday
    df["is_weekend"] = 1*(df["created_weekday"] > 5)
    
    df["total_rooms"] = df["bedrooms"] + df["bathrooms"]
    df['price_per_bed'] = df['price']/(df['bedrooms']+1)
    df['price_per_bath'] = df['price']/(df['bathrooms']+1)
    df["price_per_room"] = df['price']/(df['total_rooms']+1)
    
    df['description'] = df['description'].str.replace('<[^<>]+>', ' ')
    df['description'] = df['description'].str.replace('[0-9]+', 'num')
    df['description'] = df['description'].str.lower().replace('[^a-zA-Z0-9]', ' ')
    for i in df.index:
        df.loc[i, 'len_description'] = len(df.loc[i, 'description'])
        df.loc[i, 'num_features'] = len(df.loc[i, 'features'])

    

    if not flag:
        interest_dummies = pd.get_dummies(df.interest_level)
        df_dumm = pd.concat([df,interest_dummies[['low','medium','high']]], axis = 1).drop('interest_level', axis = 1)

        df_n = df_dumm[['building_id', 'manager_id', 'low', 'high', 'medium']]
        df_n_m = df_n.groupby(df_n.manager_id).mean()
        df_n_b = df_n.groupby(df_n.building_id).mean()

        df_n_m['manager_skill'] = df_n_m['low']*0+ df_n_m['medium']*1 + df_n_m['high']*2
        df_n_b['build_popularity'] = df_n_b['low']*0+ df_n_b['medium']*1 + df_n_b['high']*2

        df_n_m.drop(['low', 'high', 'medium'], axis=1)
        df_n_b.drop(['low', 'high', 'medium'], axis=1)
        b_dict = df_n_b.to_dict()
        m_dict = df_n_m.to_dict() 

    for i in range(df.shape[0]):
        if df.loc[df.index[i], 'building_id'] in b_dict['build_popularity']:
            df.loc[df.index[i], 'building_popularity'] = b_dict['build_popularity'][df.loc[df.index[i], 'building_id']]
        else:
            df.loc[df.index[i], 'building_popularity'] = 0.33453255354249495 #bp_mean
    
        if df.loc[df.index[i], 'manager_id'] in m_dict['manager_skill']:
            df.loc[df.index[i], 'manager_skill'] = m_dict['manager_skill'][df.loc[df.index[i], 'manager_id']]
        else:
            df.loc[df.index[i], 'manager_skill'] = 0.3666338439324288 #ms_mean
    df["logprice"] = np.log(df["price"])

    num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
    "len_description","created_year", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "price_per_bed", "price_per_bath", "total_rooms", "price_per_room",
    "manager_skill", "building_popularity", "description", "features"]
    X = df[num_feats]
    return [X, b_dict, m_dict]
train, b_dict, m_dict = feature_engineering(df=train_df,flag=0)
test, b_dict, m_dict = feature_engineering(df=test_df,flag=1, b_dict = b_dict,m_dict =  m_dict)
train["logprice"] = np.log(train_df["price"])
test["logprice"] = np.log(test_df["price"])

C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39481 entries, 10 to 99994
Data columns (total 23 columns):
bathrooms              39481 non-null float64
bedrooms               39481 non-null int64
latitude               39481 non-null float64
longitude              39481 non-null float64
price                  39481 non-null int64
num_features           39481 non-null float64
len_description        39481 non-null float64
created_year           39481 non-null int64
created_month          39481 non-null int64
created_day            39481 non-null int64
created_hour           39481 non-null int64
created_weekday        39481 non-null int64
is_night               39481 non-null int32
is_weekend             39481 non-null int32
price_per_bed          39481 non-null float64
price_per_bath         39481 non-null float64
total_rooms            39481 non-null float64
price_per_room         39481 non-null float64
manager_skill          39481 non-null float64
building_popularity    39481 non-n

In [12]:
train['description'] = train['description'].str.replace('<[^<>]+>', ' ')
train['description'] = train['description'].str.replace('[0-9]+', 'num')
train['description'] = train['description'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex=True)
test['description'] = test['description'].str.replace('<[^<>]+>', ' ')
test['description'] = test['description'].str.replace('[0-9]+', 'num')
test['description'] = test['description'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex=True)

C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [13]:
train['description'].head()

10        a brand new num bedroom num num bath apartment...
100004    top top west village location  beautiful pre w...
100007    building amenities   garage   garden   fitness...
100014                                                     
100026                                 a  website redacted 
Name: description, dtype: object

In [14]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

target_num_map = {'high':0, 'medium':2, 'low':1}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))


In [15]:
train['features'] = train_df['features']
test['features'] = test_df['features']

C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
train['features'].head()

10                                                         
100004    Laundry_In_Building Dishwasher Hardwood_Floors...
100007                               Hardwood_Floors No_Fee
100014                                                     
100026    Cats_Allowed Dogs_Allowed Elevator Laundry_In_...
Name: features, dtype: object

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39481 entries, 10 to 99994
Data columns (total 23 columns):
bathrooms              39481 non-null float64
bedrooms               39481 non-null int64
latitude               39481 non-null float64
longitude              39481 non-null float64
price                  39481 non-null int64
num_features           39481 non-null float64
len_description        39481 non-null float64
created_year           39481 non-null int64
created_month          39481 non-null int64
created_day            39481 non-null int64
created_hour           39481 non-null int64
created_weekday        39481 non-null int64
is_night               39481 non-null int32
is_weekend             39481 non-null int32
price_per_bed          39481 non-null float64
price_per_bath         39481 non-null float64
total_rooms            39481 non-null float64
price_per_room         39481 non-null float64
manager_skill          39481 non-null float64
building_popularity    39481 non-n

In [18]:
train["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

vals = train['pos'].value_counts()
dvals = vals.to_dict()
train["density"] = train['pos'].apply(lambda x: dvals.get(x, vals.min()))
test["density"] = test['pos'].apply(lambda x: dvals.get(x, vals.min()))

C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mahim\Anaconda2\envs\tensorflow-gpu-old\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39481 entries, 10 to 99994
Data columns (total 25 columns):
bathrooms              39481 non-null float64
bedrooms               39481 non-null int64
latitude               39481 non-null float64
longitude              39481 non-null float64
price                  39481 non-null int64
num_features           39481 non-null float64
len_description        39481 non-null float64
created_year           39481 non-null int64
created_month          39481 non-null int64
created_day            39481 non-null int64
created_hour           39481 non-null int64
created_weekday        39481 non-null int64
is_night               39481 non-null int32
is_weekend             39481 non-null int32
price_per_bed          39481 non-null float64
price_per_bath         39481 non-null float64
total_rooms            39481 non-null float64
price_per_room         39481 non-null float64
manager_skill          39481 non-null float64
building_popularity    39481 non-n

In [28]:
features_to_use=["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_features", 
    "len_description","created_year", "created_month", "created_day", "created_hour", "created_weekday", 
    "is_night", "is_weekend", "price_per_bed", "price_per_bath", "total_rooms", "price_per_room",
    "manager_skill", "building_popularity", "logprice", "density"]
# count_features = CountVectorizer(stop_words='english', max_features=300)
# count_features_train_sparse = count_features.fit_transform(train["features"])
# count_features_test_sparse = count_features.transform(test["features"])

# train_X = sparse.hstack([train[features_to_use], count_features_train_sparse]).tocsr()
# test_X = sparse.hstack([test[features_to_use], count_features_test_sparse]).tocsr()

# tfidf_features = TfidfVectorizer(stop_words='english', max_features=300)
# tfidf_features_train_sparse = count_features.fit_transform(train_df["features"])
# tfidf_features_test_sparse = count_features.transform(test_df["features"])

# train_X = sparse.hstack([train_X, tfidf_features_train_sparse]).tocsr()
# test_X = sparse.hstack([test_X, tfidf_features_test_sparse]).tocsr()

In [21]:
# count_desc = CountVectorizer(stop_words='english', max_features=300,ngram_range=(1,2))
# count_desc_train_sparse = count_desc.fit_transform(train_df["description"])
# count_desc_test_sparse = count_desc.transform(test_df["description"])

# train_X = sparse.hstack([train_X, count_desc_train_sparse]).tocsr()
# test_X = sparse.hstack([test_X, count_desc_test_sparse]).tocsr()

# tfidf_desc = CountVectorizer(stop_words='english', max_features=300,ngram_range=(1,2))
# tfidf_desc_train_sparse = tfidf_desc.fit_transform(train_df["description"])
# tfidf_desc_test_sparse = tfidf_desc.transform(test_df["description"])

# train_X = sparse.hstack([train_X, tfidf_desc_train_sparse]).tocsr()
# test_X = sparse.hstack([test_X, tfidf_desc_test_sparse]).tocsr()

In [27]:
print(train_X[99])

  (0, 0)	2.0
  (0, 1)	2.0
  (0, 2)	40.7779
  (0, 3)	-73.9525
  (0, 4)	6475.0
  (0, 5)	2.0
  (0, 6)	512.0
  (0, 7)	2016.0
  (0, 8)	4.0
  (0, 9)	26.0
  (0, 10)	3.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 14)	2158.33333333
  (0, 15)	2158.33333333
  (0, 16)	4.0
  (0, 17)	1295.0
  (0, 18)	0.117647058824
  (0, 19)	0.0555555555556
  (0, 20)	8.77570388657
  (0, 21)	110.0
  (0, 97)	1.0
  (0, 127)	1.0
  (0, 397)	1.0
  (0, 427)	1.0
  :	:
  (0, 1035)	1.0
  (0, 1036)	1.0
  (0, 1037)	1.0
  (0, 1042)	1.0
  (0, 1058)	1.0
  (0, 1061)	1.0
  (0, 1075)	1.0
  (0, 1076)	1.0
  (0, 1095)	1.0
  (0, 1105)	5.0
  (0, 1106)	2.0
  (0, 1110)	2.0
  (0, 1138)	1.0
  (0, 1150)	1.0
  (0, 1166)	1.0
  (0, 1167)	1.0
  (0, 1171)	1.0
  (0, 1172)	1.0
  (0, 1192)	1.0
  (0, 1199)	1.0
  (0, 1201)	1.0
  (0, 1202)	1.0
  (0, 1205)	1.0
  (0, 1211)	1.0
  (0, 1212)	1.0


In [48]:
def runXGB(param,train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [30]:
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

In [31]:
train_X = sparse.hstack([train[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test[features_to_use], te_sparse]).tocsr()

In [33]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

[0]	train-mlogloss:1.07481	test-mlogloss:1.07536
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.05077	test-mlogloss:1.05188
[2]	train-mlogloss:1.02877	test-mlogloss:1.03034
[3]	train-mlogloss:1.00742	test-mlogloss:1.00941
[4]	train-mlogloss:0.991579	test-mlogloss:0.99395
[5]	train-mlogloss:0.970617	test-mlogloss:0.973401
[6]	train-mlogloss:0.951798	test-mlogloss:0.955111
[7]	train-mlogloss:0.933147	test-mlogloss:0.936874
[8]	train-mlogloss:0.9176	test-mlogloss:0.921645
[9]	train-mlogloss:0.899848	test-mlogloss:0.904233
[10]	train-mlogloss:0.88532	test-mlogloss:0.889989
[11]	train-mlogloss:0.870536	test-mlogloss:0.875505
[12]	train-mlogloss:0.85482	test-mlogloss:0.860069
[13]	train-mlogloss:0.839835	test-mlogloss:0.845342
[14]	train-mlogloss:0.826001	test-mlogloss:0.831916
[15]	train-mlogloss:0.812827	test-mlogloss:0.819049
[16]	train-mlogloss:0.800453	test-mloglo

[155]	train-mlogloss:0.414562	test-mlogloss:0.446498
[156]	train-mlogloss:0.414109	test-mlogloss:0.446263
[157]	train-mlogloss:0.413642	test-mlogloss:0.446048
[158]	train-mlogloss:0.413216	test-mlogloss:0.445732
[159]	train-mlogloss:0.412796	test-mlogloss:0.445498
[160]	train-mlogloss:0.412438	test-mlogloss:0.44526
[161]	train-mlogloss:0.41204	test-mlogloss:0.445018
[162]	train-mlogloss:0.411626	test-mlogloss:0.444724
[163]	train-mlogloss:0.411236	test-mlogloss:0.444476
[164]	train-mlogloss:0.410859	test-mlogloss:0.444242
[165]	train-mlogloss:0.410433	test-mlogloss:0.443962
[166]	train-mlogloss:0.410017	test-mlogloss:0.443777
[167]	train-mlogloss:0.409631	test-mlogloss:0.443515
[168]	train-mlogloss:0.409266	test-mlogloss:0.443287
[169]	train-mlogloss:0.40889	test-mlogloss:0.443068
[170]	train-mlogloss:0.408547	test-mlogloss:0.442847
[171]	train-mlogloss:0.408171	test-mlogloss:0.442626
[172]	train-mlogloss:0.407816	test-mlogloss:0.44239
[173]	train-mlogloss:0.407478	test-mlogloss:0.4421

[311]	train-mlogloss:0.372143	test-mlogloss:0.429125
[312]	train-mlogloss:0.371897	test-mlogloss:0.429079
[313]	train-mlogloss:0.371691	test-mlogloss:0.429034
[314]	train-mlogloss:0.371421	test-mlogloss:0.429022
[315]	train-mlogloss:0.371265	test-mlogloss:0.429002
[316]	train-mlogloss:0.371019	test-mlogloss:0.42897
[317]	train-mlogloss:0.370751	test-mlogloss:0.428947
[318]	train-mlogloss:0.370485	test-mlogloss:0.428926
[319]	train-mlogloss:0.370297	test-mlogloss:0.428909
[320]	train-mlogloss:0.370149	test-mlogloss:0.428922
[321]	train-mlogloss:0.369915	test-mlogloss:0.428849
[322]	train-mlogloss:0.369807	test-mlogloss:0.428807
[323]	train-mlogloss:0.369485	test-mlogloss:0.428761
[324]	train-mlogloss:0.369201	test-mlogloss:0.428693
[325]	train-mlogloss:0.368985	test-mlogloss:0.428649
[326]	train-mlogloss:0.368728	test-mlogloss:0.428668
[327]	train-mlogloss:0.368523	test-mlogloss:0.428688
[328]	train-mlogloss:0.368274	test-mlogloss:0.428706
[329]	train-mlogloss:0.368075	test-mlogloss:0.4

[467]	train-mlogloss:0.339793	test-mlogloss:0.426202
[468]	train-mlogloss:0.339625	test-mlogloss:0.426203
[469]	train-mlogloss:0.339373	test-mlogloss:0.426162
[470]	train-mlogloss:0.339098	test-mlogloss:0.426165
[471]	train-mlogloss:0.338912	test-mlogloss:0.426134
[472]	train-mlogloss:0.338732	test-mlogloss:0.426141
[473]	train-mlogloss:0.338563	test-mlogloss:0.426145
[474]	train-mlogloss:0.338324	test-mlogloss:0.426139
[475]	train-mlogloss:0.338147	test-mlogloss:0.426154
[476]	train-mlogloss:0.337971	test-mlogloss:0.426165
[477]	train-mlogloss:0.337779	test-mlogloss:0.426158
[478]	train-mlogloss:0.33754	test-mlogloss:0.426206
[479]	train-mlogloss:0.33737	test-mlogloss:0.42621
[480]	train-mlogloss:0.337187	test-mlogloss:0.426231
[481]	train-mlogloss:0.336986	test-mlogloss:0.426224
[482]	train-mlogloss:0.336787	test-mlogloss:0.426211
[483]	train-mlogloss:0.336651	test-mlogloss:0.426225
[484]	train-mlogloss:0.336471	test-mlogloss:0.426216
[485]	train-mlogloss:0.336323	test-mlogloss:0.426

In [34]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39481 entries, 10 to 99994
Data columns (total 31 columns):
bathrooms              39481 non-null float64
bedrooms               39481 non-null int64
building_id            39481 non-null object
created                39481 non-null datetime64[ns]
description            39481 non-null object
display_address        39481 non-null object
features               39481 non-null object
interest_level         39481 non-null object
latitude               39481 non-null float64
listing_id             39481 non-null int64
longitude              39481 non-null float64
manager_id             39481 non-null object
photos                 39481 non-null object
price                  39481 non-null int64
street_address         39481 non-null object
created_year           39481 non-null int64
created_month          39481 non-null int64
created_day            39481 non-null int64
created_hour           39481 non-null int64
is_night               39481 no

In [35]:
test_y = np.array(test_df['interest_level'].apply(lambda x: target_num_map[x]))


In [37]:
preds, model = runXGB(train_X, train_y, test_X, test_y, num_rounds=1000)

[0]	train-mlogloss:1.07224	test-mlogloss:1.07746
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.05038	test-mlogloss:1.05873
[2]	train-mlogloss:1.02658	test-mlogloss:1.03981
[3]	train-mlogloss:1.00551	test-mlogloss:1.0221
[4]	train-mlogloss:0.983822	test-mlogloss:1.00489
[5]	train-mlogloss:0.964558	test-mlogloss:0.988868
[6]	train-mlogloss:0.944893	test-mlogloss:0.973542
[7]	train-mlogloss:0.92607	test-mlogloss:0.958928
[8]	train-mlogloss:0.9091	test-mlogloss:0.94499
[9]	train-mlogloss:0.893357	test-mlogloss:0.931669
[10]	train-mlogloss:0.878859	test-mlogloss:0.919509
[11]	train-mlogloss:0.8636	test-mlogloss:0.907545
[12]	train-mlogloss:0.85037	test-mlogloss:0.896549
[13]	train-mlogloss:0.836029	test-mlogloss:0.885715
[14]	train-mlogloss:0.822259	test-mlogloss:0.875304
[15]	train-mlogloss:0.810458	test-mlogloss:0.865611
[16]	train-mlogloss:0.797429	test-mlogloss:0

In [38]:
train_df['features'].head()

10                                                         
100004    Laundry_In_Building Dishwasher Hardwood_Floors...
100007                               Hardwood_Floors No_Fee
100014                                                     
100026    Cats_Allowed Dogs_Allowed Elevator Laundry_In_...
Name: features, dtype: object

In [39]:
tfidf_features = TfidfVectorizer(stop_words='english', max_features=200,ngram_range=(1,2))
tfidf_features_train_sparse = count_features.fit_transform(train_df["features"])
tfidf_features_test_sparse = count_features.transform(test_df["features"])

train_X = sparse.hstack([train_X, tfidf_features_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, tfidf_features_test_sparse]).tocsr()

In [40]:
count_desc = CountVectorizer(stop_words='english', max_features=300,ngram_range=(1,2))
count_desc_train_sparse = count_desc.fit_transform(train_df["description"])
count_desc_test_sparse = count_desc.transform(test_df["description"])

train_X = sparse.hstack([train_X, count_desc_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, count_desc_test_sparse]).tocsr()

tfidf_desc = CountVectorizer(stop_words='english', max_features=300,ngram_range=(1,2))
tfidf_desc_train_sparse = tfidf_desc.fit_transform(train_df["description"])
tfidf_desc_test_sparse = tfidf_desc.transform(test_df["description"])

train_X = sparse.hstack([train_X, tfidf_desc_train_sparse]).tocsr()
test_X = sparse.hstack([test_X, tfidf_desc_test_sparse]).tocsr()

In [41]:
preds, model = runXGB(train_X, train_y, test_X, test_y, num_rounds=1000)

[0]	train-mlogloss:1.07249	test-mlogloss:1.07765
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.05501	test-mlogloss:1.06113
[2]	train-mlogloss:1.03097	test-mlogloss:1.04206
[3]	train-mlogloss:1.00898	test-mlogloss:1.02459
[4]	train-mlogloss:0.98832	test-mlogloss:1.00741
[5]	train-mlogloss:0.970152	test-mlogloss:0.991915
[6]	train-mlogloss:0.951337	test-mlogloss:0.977186
[7]	train-mlogloss:0.932656	test-mlogloss:0.962739
[8]	train-mlogloss:0.917129	test-mlogloss:0.949594
[9]	train-mlogloss:0.903124	test-mlogloss:0.937048
[10]	train-mlogloss:0.887702	test-mlogloss:0.924379
[11]	train-mlogloss:0.87208	test-mlogloss:0.912106
[12]	train-mlogloss:0.857051	test-mlogloss:0.900818
[13]	train-mlogloss:0.843044	test-mlogloss:0.889804
[14]	train-mlogloss:0.829422	test-mlogloss:0.879263
[15]	train-mlogloss:0.816376	test-mlogloss:0.869281
[16]	train-mlogloss:0.803213	test-mlog

In [42]:
preds, model = runXGB(train[features_to_use], train_y, test[features_to_use], test_y, num_rounds=1000)

[0]	train-mlogloss:1.07416	test-mlogloss:1.07899
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.04909	test-mlogloss:1.05892
[2]	train-mlogloss:1.03045	test-mlogloss:1.04212
[3]	train-mlogloss:1.01081	test-mlogloss:1.02495
[4]	train-mlogloss:0.98897	test-mlogloss:1.00768
[5]	train-mlogloss:0.968571	test-mlogloss:0.991494
[6]	train-mlogloss:0.949936	test-mlogloss:0.976
[7]	train-mlogloss:0.93273	test-mlogloss:0.96163
[8]	train-mlogloss:0.914635	test-mlogloss:0.947698
[9]	train-mlogloss:0.898808	test-mlogloss:0.935096
[10]	train-mlogloss:0.883157	test-mlogloss:0.922263
[11]	train-mlogloss:0.869457	test-mlogloss:0.909816
[12]	train-mlogloss:0.856999	test-mlogloss:0.898909
[13]	train-mlogloss:0.843711	test-mlogloss:0.888242
[14]	train-mlogloss:0.832217	test-mlogloss:0.877755
[15]	train-mlogloss:0.820444	test-mlogloss:0.868219
[16]	train-mlogloss:0.806948	test-mlogloss

In [46]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

param_test1 = {
 'max_depth':[3,5,7,9],
 'min_child_weight':[1,3,5]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_log_loss',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[features_to_use],train_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -0.44302, std: 0.01075, params: {'min_child_weight': 1, 'max_depth': 3},
  mean: -0.44316, std: 0.01069, params: {'min_child_weight': 3, 'max_depth': 3},
  mean: -0.44303, std: 0.01059, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: -0.43663, std: 0.01035, params: {'min_child_weight': 1, 'max_depth': 5},
  mean: -0.43645, std: 0.01022, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: -0.43658, std: 0.01089, params: {'min_child_weight': 5, 'max_depth': 5},
  mean: -0.43940, std: 0.01092, params: {'min_child_weight': 1, 'max_depth': 7},
  mean: -0.43820, std: 0.01115, params: {'min_child_weight': 3, 'max_depth': 7},
  mean: -0.43808, std: 0.01058, params: {'min_child_weight': 5, 'max_depth': 7},
  mean: -0.45012, std: 0.01095, params: {'min_child_weight': 1, 'max_depth': 9},
  mean: -0.44619, std: 0.01115, params: {'min_child_weight': 3, 'max_depth': 9},
  mean: -0.44461, std: 0.01082, params: {'min_child_weight': 5, 'max_depth': 9}],
 {'max_depth': 5, 'min_chil

In [50]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='neg_log_loss',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[features_to_use],train_y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -0.43829, std: 0.01070, params: {'min_child_weight': 2, 'max_depth': 4},
  mean: -0.43836, std: 0.01045, params: {'min_child_weight': 3, 'max_depth': 4},
  mean: -0.43829, std: 0.01044, params: {'min_child_weight': 4, 'max_depth': 4},
  mean: -0.43644, std: 0.01078, params: {'min_child_weight': 2, 'max_depth': 5},
  mean: -0.43645, std: 0.01022, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: -0.43651, std: 0.01031, params: {'min_child_weight': 4, 'max_depth': 5},
  mean: -0.43669, std: 0.01073, params: {'min_child_weight': 2, 'max_depth': 6},
  mean: -0.43626, std: 0.01059, params: {'min_child_weight': 3, 'max_depth': 6},
  mean: -0.43646, std: 0.01053, params: {'min_child_weight': 4, 'max_depth': 6}],
 {'max_depth': 6, 'min_child_weight': 3},
 -0.43625844989011553)

In [51]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'multi:softprob', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='neg_log_loss',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[features_to_use],train_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -0.43626, std: 0.01059, params: {'gamma': 0.0},
  mean: -0.43674, std: 0.01066, params: {'gamma': 0.1},
  mean: -0.43689, std: 0.01060, params: {'gamma': 0.2},
  mean: -0.43692, std: 0.01098, params: {'gamma': 0.3},
  mean: -0.43674, std: 0.01057, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 -0.43625844989011553)

In [54]:
param = {}
param['objective'] = 'multi:softprob'
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.8
param['seed'] = 27
param['nthread'] = 4
param['gamma'] = 0
param['n_estimators'] = 140
param['scale_pos_weight'] = 1


In [57]:
# print(train_y[0:5])
preds, model = runXGB(param,train[features_to_use], train_y, test[features_to_use], test_y, num_rounds=1000)

[0]	train-mlogloss:0.872697	test-mlogloss:0.921001
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:0.733275	test-mlogloss:0.816994
[2]	train-mlogloss:0.65072	test-mlogloss:0.753863
[3]	train-mlogloss:0.594391	test-mlogloss:0.716869
[4]	train-mlogloss:0.548907	test-mlogloss:0.693768
[5]	train-mlogloss:0.518473	test-mlogloss:0.681894
[6]	train-mlogloss:0.497766	test-mlogloss:0.671128
[7]	train-mlogloss:0.479587	test-mlogloss:0.670565
[8]	train-mlogloss:0.466887	test-mlogloss:0.66863
[9]	train-mlogloss:0.457078	test-mlogloss:0.663793
[10]	train-mlogloss:0.448661	test-mlogloss:0.665566
[11]	train-mlogloss:0.441721	test-mlogloss:0.666368
[12]	train-mlogloss:0.436085	test-mlogloss:0.674671
[13]	train-mlogloss:0.431733	test-mlogloss:0.682307
[14]	train-mlogloss:0.427814	test-mlogloss:0.690913
[15]	train-mlogloss:0.424238	test-mlogloss:0.699542
[16]	train-mlogloss:0.421368	